## Parameter Visualization
- Plot each vowel and the parameterization (`LPC`/`MFCC`)
- See whether or not there exists a pattern for frontness and height to understand phonetic grounding of data

In [17]:
import matplotlib.pyplot as plt 
import scipy.io
import numpy as np

### Step 1) Import Data
- Deserialize the Matlab `.mat` files and import them as numpy arrays

In [18]:
# Parameterizations are stored as Matlab `.mat` files

# Uncomment for which parameterization to use
# PARAMETERIZATION_MATRIX_PATH = "../Matlab/data/LPC_10_vowels_236_samples.mat"
PARAMETERIZATION_MATRIX_PATH = "../Matlab/data/MFCC_10_vowels_236_samples.mat"

# Specified as the name of the matrix in the matlab processing `save` function
COEFFICIENTS_KEY = "wav_parameterization"
LABEL_KEY = "one_hot_encoding_labels"

NUM_COEFFICIENTS = 14
VOWEL_LIST = ["iy", "ih", "eh", "ae", "ey", "uw", "uh", "ow", "ao", "aa"]

In [19]:
# Import the coefficients as an n by NUM_COEFFICINETS matrix where n = number of samples
deserialized_matlab_file = scipy.io.loadmat(PARAMETERIZATION_MATRIX_PATH)

coefficient_matrix = deserialized_matlab_file[COEFFICIENTS_KEY]
coefficient_matrix = np.array(coefficient_matrix).reshape(-1, NUM_COEFFICIENTS)

labels_matrix = deserialized_matlab_file[LABEL_KEY]
labels_matrix = np.array(labels_matrix).reshape(-1, len(VOWEL_LIST))

In [20]:
# (Num Samples, NUM_COEFFICIENTS)
coefficient_matrix.shape

(236, 14)

In [21]:
# (Num Samples, Number of Vowels)
labels_matrix.shape

(236, 10)

### Step 2) Plot Data
- Plot each vowel's coefficients from [0: Number of One Hot Encodings]`